In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
#Price an ATM EuropeanCall and put using a binomial tree:

# Parameters
S0 = 100  # Initial stock price
K = 100   # Strike price (ATM)
r = 0.05  # Risk-free rate
sigma = 0.2  # Volatility
T = 0.25  # Time to maturity in years
N = 100  # Number of time steps

# Calculate the time step
dt = T / N

# Calculate the up and down factors and the probability
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u
p = (np.exp(r * dt) - d) / (u - d)

# Initialize asset prices at maturity
S = np.zeros(N + 1)
S[0] = S0 * d**N
for j in range(1, N + 1):
    S[j] = S[j - 1] * (u / d)

# Initialize option values at maturity
C = np.maximum(S - K, 0)  # Call option payoff
P = np.maximum(K - S, 0)  # Put option payoff

# Backward induction to find option prices at t = 0
for i in range(N - 1, -1, -1):
    C = np.exp(-r * dt) * (p * C[1:i + 2] + (1 - p) * C[0:i + 1])
    P = np.exp(-r * dt) * (p * P[1:i + 2] + (1 - p) * P[0:i + 1])

# Option prices at t=0
call_price = C[0]
put_price = P[0]

print(f"American Call Price: {call_price.round(2)}")
print(f"American Put Price: {put_price.round(2)}")

In [ ]:
#Compute the Greek Delta for the European call and European put at time 0:
# Parameters
S0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 0.25
N = 100
h = 0.01

def option_price_binomial(S0, K, r, sigma, T, N, option_type="call"):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    S = np.zeros(N + 1)
    S[0] = S0 * d**N
    for j in range(1, N + 1):
        S[j] = S[j - 1] * (u / d)

    if option_type == "call":
        C = np.maximum(S - K, 0)
    elif option_type == "put":
        C = np.maximum(K - S, 0)

    for i in range(N - 1, -1, -1):
        C = np.exp(-r * dt) * (p * C[1:i + 2] + (1 - p) * C[0:i + 1])

    return C[0]

# Calculate option prices for different stock prices
call_up = option_price_binomial(S0 + h, K, r, sigma, T, N, option_type="call")
call_down = option_price_binomial(S0 - h, K, r, sigma, T, N, option_type="call")

put_up = option_price_binomial(S0 + h, K, r, sigma, T, N, option_type="put")
put_down = option_price_binomial(S0 - h, K, r, sigma, T, N, option_type="put")

# Calculate Deltas
delta_call = (call_up - call_down) / (2 * h)
delta_put = (put_up - put_down) / (2 * h)



print(f"European Call Delta: {delta_call.round(2)}")
print(f"European Put Delta: {delta_put.round(2)}")


In [ ]:
#Compute the sensitivity of previous put and call option prices to a 5% increase in volatility (from 20% to 25%). How do prices change with respect to the change in volatility?
# Parameters
S0 = 100
K = 100
r = 0.05
T = 0.25
N = 100
# Function to calculate option prices using the binomial tree model
def option_price_binomial(S0, K, r, sigma, T, N, option_type="call"):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    S = np.zeros(N + 1)
    S[0] = S0 * d**N
    for j in range(1, N + 1):
        S[j] = S[j - 1] * (u / d)

    if option_type == "call":
        C = np.maximum(S - K, 0)
    elif option_type == "put":
        C = np.maximum(K - S, 0)

    for i in range(N - 1, -1, -1):
        C = np.exp(-r * dt) * (p * C[1:i + 2] + (1 - p) * C[0:i + 1])

    return C[0]


# Volatilities
sigma1 = 0.20  # Original volatility (20%)
sigma2 = 0.25  # Increased volatility (25%)

# Calculate option prices for both volatilities
call_price_sigma1 = option_price_binomial(S0, K, r, sigma1, T, N, option_type="call")
put_price_sigma1 = option_price_binomial(S0, K, r, sigma1, T, N, option_type="put")

call_price_sigma2 = option_price_binomial(S0, K, r, sigma2, T, N, option_type="call")
put_price_sigma2 = option_price_binomial(S0, K, r, sigma2, T, N, option_type="put")

# Sensitivity (Vega) due to 5% increase in volatility
vega_call = call_price_sigma2 - call_price_sigma1
vega_put = put_price_sigma2 - put_price_sigma1

print(f"call_price_sigma is 20% the price is: {call_price_sigma1.round(2)}")
print(f"call_price_sigma is 25% the price is: {call_price_sigma2.round(2)}")
print(f"vega_call: {vega_call.round(2)}")
print(f"put_price_sigma is 20% the price is: {put_price_sigma1.round(2)}")
print(f"put_price_sigma is 25% the price is: {put_price_sigma2.round(2)}")
print(f"vega_put: {vega_put.round(2)}")

In [ ]:
# Price an ATM American call and put using a binomial tree:

# Parameters
S0 = 100
K = 100
r = 0.05
sigma = 0.20
T = 0.25
N = 100


def american_option_price_binomial(S0, K, r, sigma, T, N, option_type="call"):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    discount_factor = np.exp(-r * dt)

    # Initialize asset prices at maturity
    ST = np.zeros(N + 1)
    ST[0] = S0 * d**N
    for j in range(1, N + 1):
        ST[j] = ST[j - 1] * (u / d)

    # Initialize option values at maturity
    if option_type == "call":
        option_values = np.maximum(ST - K, 0)
    elif option_type == "put":
        option_values = np.maximum(K - ST, 0)

    # Backward induction to calculate option price at time 0
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            ST[j] = ST[j] * u / d  # Moving to the next step down the tree
            continuation_value = discount_factor * (p * option_values[j + 1] + (1 - p) * option_values[j])
            exercise_value = max(ST[j] - K, 0) if option_type == "call" else max(K - ST[j], 0)
            option_values[j] = max(continuation_value, exercise_value)

    return option_values[0]


# Calculate American option prices
call_price = american_option_price_binomial(S0, K, r, sigma, T, N, option_type="call")
put_price = american_option_price_binomial(S0, K, r, sigma, T, N, option_type="put")


print(f"American Call Price: {call_price.round(2)}")
print(f"American Put Price: {put_price.round(2)}")

In [ ]:
def delta_binomial(S0, K, r, sigma, T, N, h, option_type="call", american=True):
    price_plus_h = american_option_price_binomial(S0 + h, K, r, sigma, T, N, option_type) if american else option_price_binomial(S0 + h, K, r, sigma, T, N, option_type)
    price_minus_h = american_option_price_binomial(S0 - h, K, r, sigma, T, N, option_type) if american else option_price_binomial(S0 - h, K, r, sigma, T, N, option_type)
    delta = (price_plus_h - price_minus_h) / (2 * h)
    return delta

# Parameters
S0 = 100
K = 100
r = 0.05
sigma = 0.20
T = 0.25
N = 100
h = 1  # Small change in the underlying asset price

# Compute Delta for American Call and European Put
delta_american_call = delta_binomial(S0, K, r, sigma, T, N, h, option_type="call", american=True)
delta_european_put = delta_binomial(S0, K, r, sigma, T, N, h, option_type="put", american=False)

delta_american_call, delta_european_put
print(f"American Call Delta: {delta_american_call.round(2)}")
print(f"American Put Delta: {delta_european_put.round(2)}")

In [ ]:
#Delta measures one sensitivity of the option price. The same as question 7, we will analyze the deltas for American Put and Call options here:

# Parameters
S0 = 100
K = 100
r = 0.05
T = 0.25
N = 100

# Function to compute option price given volatility
def option_price_with_volatility(S0, K, r, sigma, T, N, option_type="call", american=True):
    if american:
        return american_option_price_binomial(S0, K, r, sigma, T, N, option_type)
    else:
        return option_price_binomial(S0, K, r, sigma, T, N, option_type)


# Volatility values
sigma_20 = 0.20  # 20% volatility
sigma_25 = 0.25  # 25% volatility

# Calculate option prices at 20% volatility
call_price_sigma_20 = option_price_with_volatility(S0, K, r, sigma_20, T, N, option_type="call", american=True)
put_price_sigma_20 = option_price_with_volatility(S0, K, r, sigma_20, T, N, option_type="put", american=True)

# Calculate option prices at 25% volatility
call_price_sigma_25 = option_price_with_volatility(S0, K, r, sigma_25, T, N, option_type="call", american=True)
put_price_sigma_25 = option_price_with_volatility(S0, K, r, sigma_25, T, N, option_type="put", american=True)

# Sensitivity (Vega) due to 5% increase in volatility
vega_call = call_price_sigma_25 - call_price_sigma_20
vega_put = put_price_sigma_25 - put_price_sigma_20


print(f"call_price_sigma is 20% the price is: {call_price_sigma_20.round(2)}")
print(f"call_price_sigma is 25% the price is: {call_price_sigma_25.round(2)}")
print(f"vega_call: {vega_call.round(2)}")
print(f"put_price_sigma is 20% the price is: {put_price_sigma_20.round(2)}")
print(f"put_price_sigma is 25% the price is: {put_price_sigma_25.round(2)}")
print(f"vega_put: {vega_put.round(2)}")

In [ ]:
#Graph then show that the European call and put satisfy put-call parity using python code. Comment on the reasons why/why not the parity holds, as well as potential motives.

# Black-Scholes formula for European Call and Put options
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return price

# Parameters
S0 = 100  # Current stock price
K = 100   # Strike price
T = 1     # Time to maturity (1 year)
r = 0.05  # Risk-free interest rate (5%)
sigma = 0.2  # Volatility (20%)

# Calculate call and put prices
call_price = black_scholes(S0, K, T, r, sigma, option_type='call')
put_price = black_scholes(S0, K, T, r, sigma, option_type='put')

# Calculate the left-hand side and right-hand side of the put-call parity
lhs = call_price - put_price
rhs = S0 - K * np.exp(-r * T)

# Print the results
print(f"Call Price (C): {call_price:.2f}")
print(f"Put Price (P): {put_price:.2f}")
print(f"Left-hand side (C - P): {lhs:.2f}")
print(f"Right-hand side (S0 - K * e^(-rT)): {rhs:.2f}")

# Plotting to visualize the put-call parity
strike_prices = np.linspace(50, 150, 100)
lhs_values = []
rhs_values = []

for strike in strike_prices:
    call = black_scholes(S0, strike, T, r, sigma, option_type='call')
    put = black_scholes(S0, strike, T, r, sigma, option_type='put')
    lhs_values.append(call - put)
    rhs_values.append(S0 - strike * np.exp(-r * T))

plt.figure(figsize=(10, 6))
plt.plot(strike_prices, lhs_values, label='C - P')
plt.plot(strike_prices, rhs_values, label='S0 - K * e^(-rT)', linestyle='--')
plt.title('Put-Call Parity')
plt.xlabel('Strike Price (K)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Graph then show that the American call and put satisfy put-call parity using python code. Comment on the reasons why/why not the parity holds, as well as potential motives.

import numpy as np
import matplotlib.pyplot as plt

# Binomial tree model for American option pricing
def binomial_tree_american(S, K, T, r, sigma, steps, option_type='call'):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(steps + 1)
    ST[0] = S * (d ** steps)
    for i in range(1, steps + 1):
        ST[i] = ST[i - 1] * (u / d)

    # Initialize option values at maturity
    option_values = np.zeros(steps + 1)
    if option_type == 'call':
        option_values = np.maximum(0, ST - K)
    elif option_type == 'put':
        option_values = np.maximum(0, K - ST)

    # Backward induction to calculate the option price
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])
            ST[i] = ST[i] / d
            if option_type == 'call':
                option_values[i] = np.maximum(option_values[i], ST[i] - K)
            elif option_type == 'put':
                option_values[i] = np.maximum(option_values[i], K - ST[i])

    return option_values[0]

# Parameters
S0 = 100  # Current stock price
K = 100   # Strike price
T = 1     # Time to maturity (1 year)
r = 0.05  # Risk-free interest rate (5%)
sigma = 0.2  # Volatility (20%)
steps = 100  # Number of steps in the binomial tree

# Calculate American call and put prices
call_price_american = binomial_tree_american(S0, K, T, r, sigma, steps, option_type='call')
put_price_american = binomial_tree_american(S0, K, T, r, sigma, steps, option_type='put')

# Calculate the left-hand side and right-hand side of the modified put-call parity
lhs_american = call_price_american - put_price_american
rhs_american = S0 - K * np.exp(-r * T)

# Print the results
print(f"American Call Price (C_A): {call_price_american:.2f}")
print(f"American Put Price (P_A): {put_price_american:.2f}")
print(f"Left-hand side (C_A - P_A): {lhs_american:.2f}")
print(f"Right-hand side (S0 - K * e^(-rT)): {rhs_american:.2f}")

# Plotting to visualize the American put-call parity
strike_prices = np.linspace(50, 150, 100)
lhs_values_american = []
rhs_values_american = []

for strike in strike_prices:
    call_american = binomial_tree_american(S0, strike, T, r, sigma, steps, option_type='call')
    put_american = binomial_tree_american(S0, strike, T, r, sigma, steps, option_type='put')
    lhs_values_american.append(call_american - put_american)
    rhs_values_american.append(S0 - strike * np.exp(-r * T))

plt.figure(figsize=(10, 6))
plt.plot(strike_prices, lhs_values_american, label='C_A - P_A')
plt.plot(strike_prices, rhs_values_american, label='S0 - K * e^(-rT)', linestyle='--')
plt.title('American Put-Call Parity')
plt.xlabel('Strike Price (K)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#Select 5 strike prices so that Call options are: Deep OTM, OTM, ATM, ITM, and Deep ITM.
# Parameters
S0 = 100
r = 0.05
sigma = 0.2
T = 0.25
n = 100

# Strike prices based on moneyness
strike_prices = [0.9 * S0, 0.95 * S0, S0, 1.05 * S0, 1.1 * S0]

# Trinomial tree model for American call option pricing
def trinomial_tree_call(S, K, T, r, sigma, n):
    dt = T / n
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = 1 / u
    m = 1
    pu = ((np.exp((r - 0.5 * sigma ** 2) * dt / 2) - d) / (u - d)) ** 2
    pd = ((u - np.exp((r - 0.5 * sigma ** 2) * dt / 2)) / (u - d)) ** 2
    pm = 1 - pu - pd

    # Initialize asset prices at maturity
    ST = np.zeros((2 * n + 1,))
    ST[n] = S
    for i in range(1, n + 1):
        ST[n + i] = ST[n + i - 1] * u
        ST[n - i] = ST[n - i + 1] * d

    # Initialize option values at maturity
    option_values = np.maximum(0, ST - K)

    # Backward induction to calculate the option price
    for j in range(n - 1, -1, -1):
        for i in range(2 * j + 1):
            option_values[i] = np.exp(-r * dt) * (pu * option_values[i + 2] + pm * option_values[i + 1] + pd * option_values[i])

    return option_values[0]

# Calculate call option prices for the selected strike prices
call_prices = [trinomial_tree_call(S0, K, T, r, sigma, n) for K in strike_prices]

# Output the results
for K, price in zip(strike_prices, call_prices):
    print(f"Strike Price (K) = {K:.2f}, Call Option Price = {price:.2f}")


In [ ]:


# Parameters
r = 0.05      # Risk-free interest rate (5%)
sigma = 0.2   # Volatility (20%)
T = 1         # Time to maturity (1 year)
steps = 100   # Number of steps for the Binomial Tree model

# Black-Scholes formula for European Call options
def black_scholes_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

# Black-Scholes formula for European Put options
def black_scholes_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return put_price

# Binomial tree model for American call option pricing
def binomial_tree_american_call(S, K, T, r, sigma, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(steps + 1)
    ST[0] = S * (d ** steps)
    for i in range(1, steps + 1):
        ST[i] = ST[i - 1] * (u / d)

    # Initialize option values at maturity
    option_values = np.maximum(0, ST - K)

    # Backward induction to calculate the option price
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])
            ST[i] = ST[i] / d
            option_values[i] = np.maximum(option_values[i], ST[i] - K)

    return option_values[0]

# Binomial tree model for American put option pricing
def binomial_tree_american_put(S, K, T, r, sigma, steps):
    dt = T / steps
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(steps + 1)
    ST[0] = S * (d ** steps)
    for i in range(1, steps + 1):
        ST[i] = ST[i - 1] * (u / d)

    # Initialize option values at maturity
    option_values = np.maximum(0, K - ST)

    # Backward induction to calculate the option price
    for j in range(steps - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])
            ST[i] = ST[i] / d
            option_values[i] = np.maximum(option_values[i], K - ST[i])

    return option_values[0]

# Parameters for the graphs
stock_prices = np.linspace(50, 150, 100)  # Stock prices from 50 to 150
strike_prices = np.linspace(80, 120, 100)  # Strike prices from 80 to 120
K_fixed = 100  # Fixed strike price for graphs 1 and 2
S_fixed = 100  # Fixed stock price for graphs 3 and 4

# Lists to store option prices
european_call_prices_vs_stock = []
european_put_prices_vs_stock = []
american_call_prices_vs_stock = []
american_put_prices_vs_stock = []

european_call_prices_vs_strike = []
american_call_prices_vs_strike = []
european_put_prices_vs_strike = []
american_put_prices_vs_strike = []

# Calculate option prices for varying stock prices (Graph #1 and #2)
for S in stock_prices:
    european_call_prices_vs_stock.append(black_scholes_call(S, K_fixed, T, r, sigma))
    european_put_prices_vs_stock.append(black_scholes_put(S, K_fixed, T, r, sigma))
    american_call_prices_vs_stock.append(binomial_tree_american_call(S, K_fixed, T, r, sigma, steps))
    american_put_prices_vs_stock.append(binomial_tree_american_put(S, K_fixed, T, r, sigma, steps))

# Calculate option prices for varying strike prices (Graph #3 and #4)
for K in strike_prices:
    european_call_prices_vs_strike.append(black_scholes_call(S_fixed, K, T, r, sigma))
    american_call_prices_vs_strike.append(binomial_tree_american_call(S_fixed, K, T, r, sigma, steps))
    european_put_prices_vs_strike.append(black_scholes_put(S_fixed, K, T, r, sigma))
    american_put_prices_vs_strike.append(binomial_tree_american_put(S_fixed, K, T, r, sigma, steps))

# Plot Graph #1: European call prices and put prices versus stock prices
plt.figure(figsize=(12, 6))
plt.plot(stock_prices, european_call_prices_vs_stock, label='European Call Prices')
plt.plot(stock_prices, european_put_prices_vs_stock, label='European Put Prices')
plt.title('European Call and Put Prices vs Stock Prices')
plt.xlabel('Stock Price (S)')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()

# Plot Graph #2: American call prices and put prices versus stock prices
plt.figure(figsize=(12, 6))
plt.plot(stock_prices, american_call_prices_vs_stock, label='American Call Prices')
plt.plot(stock_prices, american_put_prices_vs_stock, label='American Put Prices')
plt.title('American Call and Put Prices vs Stock Prices')
plt.xlabel('Stock Price (S)')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()

# Plot Graph #3: European and American call prices versus strike prices
plt.figure(figsize=(12, 6))
plt.plot(strike_prices, european_call_prices_vs_strike, label='European Call Prices')
plt.plot(strike_prices, american_call_prices_vs_strike, label='American Call Prices')
plt.title('European and American Call Prices vs Strike Prices')
plt.xlabel('Strike Price (K)')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()

# Plot Graph #4: European and American put prices versus strike prices
plt.figure(figsize=(12, 6))
plt.plot(strike_prices, european_put_prices_vs_strike, label='European Put Prices')
plt.plot(strike_prices, american_put_prices_vs_strike, label='American Put Prices')
plt.title('European and American Put Prices vs Strike Prices')
plt.xlabel('Strike Price (K)')
plt.ylabel('Option Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#Check whether put-call parity holds (within sensible rounding). Briefly comment on the reasons why/why not this is the case.

# Parameters
S = 100
r = 0.05
T = 0.25

# Strike prices and call option prices
strike_prices = np.array([90, 95, 100, 105, 110])
call_prices = np.array([10.47, 6.72, 3.87, 2.00, 0.92])

# Calculate the present value of strike prices
K_discounted = strike_prices * np.exp(-r * T)

# Calculate theoretical put prices using put-call parity: P = C + K_discounted - S
theoretical_put_prices = call_prices + K_discounted - S

# Print results
for i in range(len(strike_prices)):
    print(f"Strike Price (K): {strike_prices[i]}, Call Price (C): {call_prices[i]:.2f}, "
          f"Theoretical Put Price (P): {theoretical_put_prices[i]:.2f}")
